## 13.1 Dagger-Hilt - Podstawy

Wv tej aplikacji zapoznamy się z podstawami wykorzystania `Dagger-Hilt` w aplikacji. Jak zobaczymy zastosowanie biblioteki `Hilt` mocno upraszcza całą procedurę *wstrzyknięć*. Wykorzystamy `Retrofit2`, `LoggingInterceptor` i architekturę **MVVM**. Jako backend ponownie wybierzemy https://jsonplaceholder.typicode.com/.

Rozpocznijmy od dodania odpowiednich zależności

In [ ]:
plugins {
    id 'com.android.application'
    id 'org.jetbrains.kotlin.android'
    id 'kotlin-kapt'
    id 'dagger.hilt.android.plugin'
}
...
dependencies {

    implementation "com.google.dagger:hilt-android:2.44"
    kapt "com.google.dagger:hilt-android-compiler:2.44"
    kapt "androidx.hilt:hilt-compiler:1.0.0"

    // activity dla kotlin KTX + ViewModel
    implementation "androidx.activity:activity-ktx:1.5.0"
    
    implementation 'com.squareup.retrofit2:retrofit:2.9.0'
    implementation 'com.google.code.gson:gson:2.9.1'
    implementation 'com.squareup.retrofit2:converter-gson:2.9.0'
    implementation 'com.squareup.okhttp3:logging-interceptor:4.10.0'
}

Do bloku `plugins` pliku `build.gradle(Project)` dodajemy

In [ ]:
id 'com.google.dagger.hilt.android' version '2.44' apply false

### Szkielet projektu

Naszą aplikację podzielimy na kilka pakietów
- `data` - model danych + dwa pakiety
    - `remote` zawierający interfejs `PlaceholderApi`
    - `repository` zawierający implementację repozytorium aplikacji
- `di` - przeznaczony do *dependency injection* - tutaj dodamy moduły
- `domain/remote` - zawiera interfejs repozytorium
- `ui` - elementy interfejsu użytkownika
- `util` - tutaj umieścimy znaną z poprzednich przykładów klasę `Resource`
- w głównym pakiecie znajduje się klasa `MyApp` rozszerzająca `Application`

Rozpocznijmy od interfejsu `PlaceholderApi` w pakiecie `data/remote`. Jest on identyczny jak w przykładzie 11.1

In [ ]:
interface PlaceholderApi {
    @GET("posts")
    suspend fun posts(): Response<List<Post>>
}

Dodajmy interfejs `AppRepository` dodający poziom abstrakcji do pakietu `domain/remote`

In [ ]:
interface AppRepository {
    suspend fun getPosts(): Response<List<Post>>
}

Do pakietu `data` dodajmy model danych (identyczny jak w 11.1)

In [ ]:
data class Post (
    val userId: Int,
    val id: Int,
    val title: String,

    @SerializedName("body")
    val content: String
)

Do pakietu `data/repository` dodajmy klasę `AppRepositoryImpl` implementującą interfejs `AppRepository`

In [ ]:
class AppRepositoryImpl(
    private val api: PlaceholderApi
) : AppRepository {
    override suspend fun getPosts() = api.posts()
}

Do pakietu `di` dodajmy nasz moduł `AppModule`, tutaj zdefiniujemy instancję `Retrofit`. Chcemy posiadać jeden moduł o czasie życia aplikacji (powiązany z kontekstem `Application`), więc będzie to `object`

In [ ]:
@Module
object AppModule {}

Drugą adnotacją którą zastosujemy jest `@InstallIn` - deklaruje do których komponentów opisana klasa/obiekt powinna zostać dodana podczas generacji obiektów przez `Hilt`.

In [ ]:
@Module
@InstallIn(SingletonComponent::class)
object AppModule {}

Komponent podany jako argument `@InstallIn` decyduje o czasie życia zależności dodanych w module. Niektóre przekłady
- `SingletonComponent` - czas życia aplikacji (poprzednio nazywany `ApplicationComponent`)
- `ActivityComponent` - czas życia aktywności
- `ViewModelComponent` - czas życia `ViewModel`
- `ServiceComponent` - czas życia serwisu
- `ActivityRetainedComponent` - czas życia aktywności + zmiana konfiguracji (rotacja)

Dodajmy metodę `providePlaceholderApi`, gdzie utworzymy instancję `Retrofit` i dodamy `LoggingInterceptor`

In [ ]:
@Provides
@Singleton
fun providePlaceholderApi(): PlaceholderApi{
    val interceptor = HttpLoggingInterceptor()
    interceptor.setLevel(HttpLoggingInterceptor.Level.BODY)
    val client = OkHttpClient.Builder()
        .addInterceptor(interceptor)
        .build()
    return Retrofit.Builder()
        .baseUrl("https://jsonplaceholder.typicode.com/")
        .addConverterFactory(GsonConverterFactory.create())
        .client(client)
        .build().create(PlaceholderApi::class.java)
}

Jedynym nowym elementem jest adnotacja `@Singleton` - jeżeli `PlaceholderApi` zostanie wstrzyknięty do kilku klas, będzie to ta sama instancja. Bez tej adnotacji, przy wstrzykiwaniu do kilku klas, za każdym razem tworzona jest nowa instancja.

Dodajmy drugą metodę dostarczającą repozytorium.

In [ ]:
@Provides
@Singleton
fun provideAppRepository(api: PlaceholderApi) : AppRepository{
    return AppRepositoryImpl(api)
}

Do pakietu `util` dodajmy, znaną w wcześniejszych przykładów, klasę `Resource`

In [ ]:
sealed class Resource<T> (
    val data: T? = null,
    val message: String? = null
){
    class Success<T>(data: T) : Resource<T>(data)
    class Error<T>(message: String, data: T? = null) : Resource<T>(data, message)
    class Loading<T> : Resource<T>()
}

Dodajmy `AppViewModel`, wykorzystamy adnotację `HiltViewModel`, który pozwala bibliotece `Hilt` wykonać wstrzyknięcia do `ViewModel`, oraz wstrzyknąć sam `ViewModel` (jak i zastosować w kotlinie `by viewmodel()`) - dzięki tej adnotacji jest to o wiele łatwiejszev niż w przypadku zastosowania biblioteki `Dagger2`.

Wykorzystamy adnotację `@Inject` do wstrzyknięcia przez konstruktor repozytorium do `AppViewModel`.

In [ ]:
@HiltViewModel
class AppViewModel @Inject constructor (
    private val repository: AppRepository
) : ViewModel() {

Dodajmy listę wszystkich postów.

In [ ]:
private var _posts: MutableLiveData<Resource<List<Post>>> = MutableLiveData()

val posts: LiveData<Resource<List<Post>>>
    get() = _posts

Metodę obsługującą odpowiedź.

In [ ]:
private fun handleMealResponse(response: Response<List<Post>>)
        : Resource<List<Post>> {
    if (response.isSuccessful)
        response.body()?.let { return Resource.Success(it) }
    return Resource.Error(response.message())
}

Na koniec dodajmy metodę `getPosts` wykorzystując `Coroutines` i `viewModelScope`

In [ ]:
fun getPosts() = viewModelScope.launch {
    _posts.postValue(Resource.Loading())
    val response = repository.getPosts()
    _posts.postValue(handleMealResponse(response))
}

Jak widzimy, poza zastosowaniem kilku adnotacji w samym kodzie nic się nie zmienia.

Wymaganym jest również dodanie klasy rozszerzającej klasę `Application`, z adnotacją `HiltAndroidApp`

In [ ]:
@HiltAndroidApp
class MyApp : Application() {
}

Dzięki temu `Hilt` *wie* że w tej klasie może tworzyć komponenty `Dagger2`. Adnotacja `@HiltAndroidApp` wyzwala generowanie komponentów `Dagger2` przez `Hilt`. Tutaj musimy jeszcze dokonać zmiany w pliku `AndroidManifest` i dodać `name` o nazwie klasy rozszerzającej `Application`

In [ ]:
<application
    android:name=".MyApp"
    ...

Ta klasa może zostać wykorzystana w momencie gdy musimy dostarczyć `applicationContext` (np. do `ROOM`) - `Hilt` automatycznie wygeneruje odpowiedni kod.

W aktywności głównej dodajemy `ViewModel`

In [ ]:
private val viewModel: AppViewModel by viewModels()

W samej aktywności musimy dodać adnotację `@AndroidEntryPoint` - wskazuje punkt wejściowy aplikacji dla `Hilt`

W metodzie `onCreate` inicjujemy połączenie z serwerem.

In [ ]:
viewModel.getPosts()

W pozostałych elementach nie ma żadnych nowych elementów.

In [ ]:
    private val viewModel: AppViewModel by viewModels()
    private val progressBar: ProgressBar by lazy{findViewById(R.id.progressBar)}

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_main)

        viewModel.getPosts()

        val textView = findViewById<TextView>(R.id.textView)

        viewModel.posts.observe(this){ response ->
            when (response) {
                is Resource.Success -> {
                    hideProgressBar()
                    response.data?.let { res ->
                        val content = StringBuilder()
                        res.forEach {
                            content
                                .append("id: ").append(it.id).append("\n")
                                .append("UserId: ").append(it.userId).append("\n")
                                .append("title: ").append(it.title).append("\n")
                                .append("body: ").append(it.content).append("\n\n")
                        }
                        textView.text = content
                    }
                }
                is Resource.Error -> {
                    hideProgressBar()
                    response.message?.let { Log.e("TAG", "Error occurred: $it") }
                }
                is Resource.Loading -> showProgressBar()
            }
        }
    }

    private fun hideProgressBar(){
        progressBar.visibility = View.INVISIBLE
    }

    private fun showProgressBar(){
        progressBar.visibility = View.VISIBLE
    }

g